In [1]:
# imports
import os
import re
import requests
import urllib
import glob
import pandas as pd
from bs4 import BeautifulSoup

# Variables

folder = "Covid_Daily_Cases"
url = "https://lua.rlp.de/de/unsere-themen/infektionsschutz/meldedaten-coronavirus/meldedaten-excel/"
# headers
columns_1 = [('district',''), 
             ('Laboratory-confirmed','total'),
             ('Laboratory-confirmed','difference_previous_day'),
             ('Laboratory-confirmed','hospitalized'),
             ('Laboratory-confirmed','deceased'),
             ('Laboratory-confirmed','recovered'),
             ('Laboratory-confirmed','current_cases'),
             ('New_reporting_cases','Last_7_days'),
             ('New_reporting_cases','same_period_previous_week'),
             ('Incidence_last_7_days_per_100,000','Rhineland_Palatinate'),
             ('Incidence_last_7_days_per_100,000','USAF'),
             ('Incidence_last_7_days_per_100,000','yr_20_less'),
             ('Incidence_last_7_days_per_100,000','yr_20_59'),
             ('Incidence_last_7_days_per_100,000','yr_60_above'),
             ('Incidence_last_7_days_per_100,000','hospitalization_in_rlp')
            ]

columns_2 = [('district', ''),
             ('Laboratory-confirmed', 'total'),
             ('Laboratory-confirmed', 'difference_previous_day'),
             ('Laboratory-confirmed', 'hospitalized'),
             ('Laboratory-confirmed', 'deceased'),
             ('Laboratory-confirmed', 'recovered'),
             ('Laboratory-confirmed', 'current_cases'),
             ('Incidence_last_7_days_per_100,000', 'Total'),
             ('Incidence_last_7_days_per_100,000', 'Rhineland_Palatinate'),
             ('Incidence_last_7_days_per_100,000', 'USAF'),
             ('Incidence_last_7_days_per_100,000', 'yr_20_less'),
             ('Incidence_last_7_days_per_100,000', 'yr_20_59'),
             ('Incidence_last_7_days_per_100,000', 'yr_60_above')
]

path = '/Users/mayankrupareliya/EDA/Covid_Daily_Cases/'
extension = 'xlsx'

In [2]:
# Variables

folder = "Covid_Daily_Cases"
url = "https://lua.rlp.de/de/unsere-themen/infektionsschutz/meldedaten-coronavirus/meldedaten-excel/"
# headers
columns_1 = [('district',''), 
             ('total'),
             ('difference_previous_day'),
             ('hospitalized'),
             ('deceased'),
             ('recovered'),
             ('current_cases'),
             ('Last_7_days'),
             ('same_period_previous_week'),
             ('Rhineland_Palatinate'),
             ('USAF'),
             ('yr_20_less'),
             ('yr_20_59'),
             ('yr_60_above'),
             ('hospitalization_in_rlp')
            ]

columns_2 = [('district', ''),
             ('total'),
             ('difference_previous_day'),
             ('hospitalized'),
             ('deceased'),
             ('recovered'),
             ('current_cases'),
             ('Total'),
             ('Rhineland_Palatinate'),
             ('USAF'),
             ('yr_20_less'),
             ('yr_20_59'),
             ('yr_60_above')
]

path = '/Users/mayankrupareliya/EDA/Covid_Daily_Cases/'
extension = 'xlsx'

In [3]:
def extract_excel_files(url, folder):
    # Create if folder doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Get content of Webpage
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
        else:
            print(f"Error: Status code {response.status_code}")
    except Exception as e:
        print(e)

    # Find all links to Excel files
    excel_links = ["https://lua.rlp.de/" + str(a["href"]) for a in soup.find_all("a", href=True) if a["href"].endswith(".xlsx")]
    
    
    # Iterate through the links to download files
    for link in excel_links:
        file_name = link.split("/")[-1]
        file_path = os.path.join(folder, file_name)
        if not os.path.exists(file_path):
            print(f"Downloading {file_name}")
            urllib.request.urlretrieve(link, file_path)
        else:
            print(f"Skipping {file_name}")


In [4]:
extract_excel_files(url, folder)

In [5]:
def merge_files(files):
    final_data_13_columns = pd.DataFrame()                       # empty dataframe      
    final_data_15_columns = pd.DataFrame()                       # empty dataframe 
    for file in files:
        filepath = path + file                                       # file_path       
        date = re.findall('[A-Z]-(.*).xlsx',file)[0]                                    # extract date variable 
        df = pd.read_excel(open(filepath, 'rb'), sheet_name='Tabelle1', header=None)    # read excel
    
        df = df.iloc[3:-1]                                                              # Drop the first 3 rows and last row
        df = df.drop(df.columns[-1], axis=1)                                            # Drop the last column
        df = df.reset_index(drop=True)                                                  # Reset the index
        
        if len(df.columns) == 13:
            df.columns=pd.MultiIndex.from_tuples(columns_2)                             # add headers to dataframe
            df['date'] = date                                                           # add date as column
            final_data_13_columns = pd.concat([final_data_13_columns, df], ignore_index=True)      # merging into single dataframe
        elif len(df.columns) == 15:
            df.columns=pd.MultiIndex.from_tuples(columns_1)                             # add headers to dataframe
            df['date'] = date                                                           # add date as column
            final_data_15_columns = pd.concat([final_data_15_columns, df], ignore_index=True)      # merging into single dataframe
            
    final_data_13_columns.to_excel("COVID_case_report_1.xlsx", sheet_name='Table 1') 
    final_data_15_columns.to_excel("COVID_case_report_2.xlsx", sheet_name='Table 1') 
    print("Files Merged")

In [6]:
os.chdir(path)
files = glob.glob('*.{}'.format(extension))
merge_files(files)

Files Merged
